# MAAP Benchmark with "get-dem" 

## Setting up environment

In [ ]:
import os
from time import time
import subprocess
import numpy as np

# Uncomment to install sardem
#!pip install sardem
os.environ['PROJ_LIB']='/opt/conda/share/proj'

## Setup test

In [ ]:
bboxes = ['-156 18.8 -154.7 20.3', # little area
    '-118.06817 34.22169 -118.05801 34.22822',  # Mt Wilson (near JPL) (small, ~8 sec)
          '-124.81360 32.44506 -113.75989 42.24498', # CA and NV (uses ~5-6 GB RAM, takes a few minutes to run)

           # Very large bbox options -- might run out of memory
        '-125.53514 25.42150  -87.06960 49.22105', # Western US (CA, WA, Chicago, New Orleans)
        '-125.41615 31.03621 -101.80916 49.17324', # Western US (CA -> CO)
        '6.26868 36.00380   18.57179 47.28139'  # Italy (Requires more than 8 GB RAM)
         ]



bbox_index = 1
mc_count = 20

## ADE Benchmark

Launching the "get-dem" multiple time. 
* Make sure that there are no errors in the "compute" part when running the command line *

In [ ]:
exec_time_list = []
start = time()

i=0
while i<mc_count:
    start = time()
    !sh /projects/get-dem/driver/run.sh -b {bboxes[bbox_index]} --compute TRUE 2>&1 > /dev/null
    exec_time_list.append(time()-start)
    !rm /projects/data/output/dem.tif
    i = i+1


In [ ]:
print("Mean execution time :"+ str(np.mean(exec_time_list)))
print("Mean execution time :"+ str(np.std(exec_time_list)))

## WPST Benchmark

In [ ]:
from maap_utils.maap_process import MaapWPST
from maap_utils.maap_authenticator import MaapAuthenticator

In [ ]:
maap_auth = MaapAuthenticator('/projects/.maap/auth.ini', '/projects/.maap/maap.ini')
oauth_token = maap_auth.get_esa_token_with_esa_cred()

maap_wpst = MaapWPST('/projects/.maap/maap.ini', oauth_token)

In [ ]:
i=0
while i< mc_count:
    i = i+1
    maap_job = maap_wpst.launch_process(
        'wf-get-dem',
        [{"id":"s_bbox","data":bboxes[bbox_index],"href":""},
         {"id":"s_compute","data":"TRUE","href":""},
         {'id': 'copy_dir_or_file', 'data': 'dir', 'href': ''},
         {'id': 's3_destination', 'data': '', 'href': 'maap-scientific-data/shared/adu/todelete/benchmark/getdem'},
         {'id': 'source_folder', 'data': '/projects/data/output', 'href': ''}]
    )
    #maap_wpst.wait_for_final_status(maap_job)
    #maap_wpst.delete_job(maap_job)
    